In [9]:
import vincent
import pandas as pd
vincent.core.initialize_notebook()


In [10]:
import json
with open('us_counties.topo.json', 'r') as f:
    get_id = json.load(f)

#A little FIPS code munging
new_geoms = []
for geom in get_id['objects']['us_counties.geo']['geometries']:
    geom['properties']['FIPS'] = int(geom['properties']['FIPS'])
    new_geoms.append(geom)

get_id['objects']['us_counties.geo']['geometries'] = new_geoms

In [11]:
with open('us_counties.topo.json', 'w') as f:
    json.dump(get_id, f)

In [6]:
geometries = get_id['objects']['us_counties.geo']['geometries']
county_codes = [x['properties']['FIPS'] for x in geometries]
county_df = pd.DataFrame({'FIPS': county_codes}, dtype=str)
county_df = county_df.astype(int)
county_topo = r'us_counties.topo.json'

In [7]:
df = pd.read_csv(r'/IDS/data/interested.csv', na_values=[' '])
df['FIPS_Code'] = df['FIPS'].astype(str)


In [8]:
merged = pd.merge(df, county_df, on='FIPS', how='outer')
# print(merged)
# merged = merged.fillna(method='pad')

# merged['poverty'] = merged.apply(lambda row: 1-row['poverty'],axis=1)
# merged['bool'] = merged.apply(lambda row: row['FIPS'] in df['FIPS'],axis=1)
# print(merged)

In [50]:
geo_data = [{'name': 'counties',
             'url': county_topo,
             'feature': 'us_counties.geo'}]

vis = vincent.Map(data=merged, geo_data=geo_data, scale=1100, projection='albersUsa',
          data_bind='bool', data_key='FIPS',brew='YlOrRd',
          map_key={'counties': 'properties.FIPS'})

In [51]:

vis.to_json('vega.json')

In [52]:
vis.display()